In [1]:
import pandas as pd
import numpy as np
import json
import xgboost as xgb

In [12]:
timestamp = pd.Timestamp('2015-05-01 00:00:00')

In [49]:
departures_model = xgb.Booster(model_file='../models/boosterDepartures.xgbm')
arrivals_model = xgb.Booster(model_file='../models/boosterArrivals.xgbm')

#load additional dfeatures
additional_features_df = pd.read_csv('../data/modelInput/additionalFeatures.csv')
additional_features_df = additional_features_df.drop(['icon', 'precipType', 'summary', 'date', 'holiday_description'], axis=1)
additional_features_df['date_hour'] = additional_features_df['date_hour'].apply(pd.Timestamp)
additional_features_df = additional_features_df.set_index('date_hour')
additional_features_df = additional_features_df.tz_localize(None)

In [50]:
additional_features_df.head()

,apparentTemperature,cloudCover,dewPoint,humidity,precipAccumulation,precipIntensity,precipProbability,pressure,temperature,uvIndex,visibility,windBearing,windSpeed,hour,weekday,month,year,is_holiday,is_weekend,is_weekend_or_holiday
date_hour,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,-10.83,0.22,-11.59,0.63,NaN,0.0,0.0,1019.96,-5.76,0.0,15.40,229,3.42,0,3,1,2015,1,0,1
2015-01-01 01:00:00,-11.83,0.17,-12.09,0.62,NaN,0.0,0.0,1019.03,-5.94,0.0,15.26,226,4.31,1,3,1,2015,1,0,1
2015-01-01 02:00:00,-12.07,0.06,-12.44,0.60,NaN,0.0,0.0,1018.15,-6.03,0.0,15.48,232,4.47,2,3,1,2015,1,0,1
2015-01-01 03:00:00,-11.53,0.07,-12.67,0.57,NaN,0.0,0.0,1017.48,-5.43,0.0,15.42,236,4.74,3,3,1,2015,1,0,1
2015-01-01 04:00:00,-11.01,0.34,-12.96,0.53,NaN,0.0,0.0,1016.75,-4.78,0.0,15.51,242,5.17,4,3,1,2015,1,0,1


In [65]:
cluster_number = 50
with open('../models/stationClusterMap.json', 'r') as f:
     cluster_station_map = json.load(f)

In [52]:
stations = pd.read_csv('../data/modelInput/stations.csv')
stations['first_used'] = stations['first_used'].apply(pd.Timestamp)
stations['last_used'] = stations['last_used'].apply(pd.Timestamp)
stations.head()

,station_id,station_name,latitude,longitude,first_used,last_used
0,3,Colleges of the Fenway,42.340021,-71.100812,2015-04-17,2016-11-30
1,4,Tremont St. at Berkeley St.,42.345392,-71.069616,2015-01-02,2016-11-30
2,5,Northeastern U / North Parking Lot,42.341814,-71.090179,2015-04-17,2016-11-30
3,6,Cambridge St. at Joy St.,42.361174,-71.065142,2015-01-01,2016-11-30
4,7,Fan Pier,42.353287,-71.044389,2015-04-17,2016-11-30


In [56]:
clusters = np.arange(0, cluster_number)
timestamp_start = pd.Timestamp(timestamp.year, timestamp.month, timestamp.day, 0)
timestamp_end = pd.Timestamp(timestamp.year, timestamp.month, timestamp.day, 23)
date_hours = pd.date_range(timestamp_start, timestamp_end, freq='H')
cluster_time_index = pd.MultiIndex.from_product([date_hours, clusters], names=['date_hour', 'cluster_id'])
cluster_time_df = pd.DataFrame({'arrivals': 0, 'departures': 0}, index=cluster_time_index)
cluster_time_df = cluster_time_df.reset_index()
cluster_time_df = cluster_time_df.set_index('date_hour')
#cluster_time_df = cluster_time_df.tz_localize('EST')

In [57]:
cluster_time_df.head()

,cluster_id,arrivals,departures
date_hour,,,
2015-05-01,0,0,0
2015-05-01,1,0,0
2015-05-01,2,0,0
2015-05-01,3,0,0
2015-05-01,4,0,0


In [58]:
model_input = cluster_time_df.merge(additional_features_df, how='left', left_index=True, right_index=True)
#model_input = model_input.tz_convert('EST')
#model_input = model_input.tz_localize(None)
model_input.head()

,cluster_id,arrivals,departures,apparentTemperature,cloudCover,dewPoint,humidity,precipAccumulation,precipIntensity,precipProbability,...,visibility,windBearing,windSpeed,hour,weekday,month,year,is_holiday,is_weekend,is_weekend_or_holiday
date_hour,,,,,,,,,,,,,,,,,,,,,
2015-05-01,0,0,0,5.9,0.16,3.72,0.8,NaN,0.0,0.0,...,15.51,64,1.73,0,4,5,2015,0,0,0
2015-05-01,1,0,0,5.9,0.16,3.72,0.8,NaN,0.0,0.0,...,15.51,64,1.73,0,4,5,2015,0,0,0
2015-05-01,2,0,0,5.9,0.16,3.72,0.8,NaN,0.0,0.0,...,15.51,64,1.73,0,4,5,2015,0,0,0
2015-05-01,3,0,0,5.9,0.16,3.72,0.8,NaN,0.0,0.0,...,15.51,64,1.73,0,4,5,2015,0,0,0
2015-05-01,4,0,0,5.9,0.16,3.72,0.8,NaN,0.0,0.0,...,15.51,64,1.73,0,4,5,2015,0,0,0


In [121]:
x_model_input = xgb.DMatrix(model_input)
model_input['departures'] = departures_model.predict(x_model_input)
model_input['arrivals'] = arrivals_model.predict(x_model_input)
model_input = model_input.reset_index()
model_input.head()

,date_hour,cluster_id,arrivals,departures,apparentTemperature,cloudCover,dewPoint,humidity,precipAccumulation,precipIntensity,...,visibility,windBearing,windSpeed,hour,weekday,month,year,is_holiday,is_weekend,is_weekend_or_holiday
0,2015-05-01,0,0.666281,0.407924,5.9,0.16,3.72,0.8,NaN,0.0,...,15.51,64,1.73,0,4,5,2015,0,0,0
1,2015-05-01,1,1.177789,0.772143,5.9,0.16,3.72,0.8,NaN,0.0,...,15.51,64,1.73,0,4,5,2015,0,0,0
2,2015-05-01,2,0.360593,0.259184,5.9,0.16,3.72,0.8,NaN,0.0,...,15.51,64,1.73,0,4,5,2015,0,0,0
3,2015-05-01,3,0.174178,0.083475,5.9,0.16,3.72,0.8,NaN,0.0,...,15.51,64,1.73,0,4,5,2015,0,0,0
4,2015-05-01,4,0.025554,0.024951,5.9,0.16,3.72,0.8,NaN,0.0,...,15.51,64,1.73,0,4,5,2015,0,0,0


In [122]:
active_stations = stations[(stations['first_used'] < timestamp) & (stations['last_used'] > timestamp)]['station_id'].unique()

In [134]:
weight_matrix = np.matlib.zeros((cluster_number, active_stations.size))
for cluster_index in range(0, cluster_number):
    cluster_stations = cluster_station_map[str(cluster_index)]
    station_index = 0
    for station_id in active_stations:
        if (str(station_id) in cluster_stations):
            weight_matrix[cluster_index, station_index] = cluster_stations[str(station_id)]
        station_index += 1

inv_weight_matrix = weight_matrix.I
weight_threshold = 1e-3
trips = pd.DataFrame([], columns=['date_hour', 'station_id', 'arrivals', 'departures'])
for cluster_index in range(0, cluster_number):
    station_index = 0
    for station_id in active_stations:
        weight = inv_weight_matrix[station_index, cluster_index] 
        if(weight > weight_threshold):
            weighted_trips = model_input[model_input['cluster_id'] == cluster_index][['date_hour', 'arrivals', 'departures']]
            weighted_trips['arrivals'] = weighted_trips['arrivals'] * weight
            weighted_trips['departures'] = weighted_trips['departures'] * weight
            weighted_trips['station_id'] = station_id
            trips = trips.append(weighted_trips)
        station_index += 1

trips = trips.groupby(['date_hour', 'station_id']).sum()
trips['flow'] = trips['arrivals'] - trips['departures']

In [135]:
trips

arrivals  departures      flow
date_hour           station_id                                
2015-05-01 00:00:00 3           0.076614    0.050039  0.026575
                    4           0.121665    0.068767  0.052898
                    5           0.143325    0.102205  0.041121
                    6           0.115175    0.070666  0.044509
                    7           0.056254    0.056898 -0.000644
                    8           0.064178    0.030524  0.033654
                    9           0.453147    0.258992  0.194155
                    10          0.141177    0.091699  0.049478
                    11          0.076645    0.050025  0.026620
                    12          0.147554    0.105220  0.042334
                    13          0.107944    0.065675  0.042269
                    14          0.076316    0.049812  0.026504
                    15          0.064375    0.030618  0.033758
                    16          0.096290    0.058953  0.037337
                    17          0.169623    0.101865  0.067757
                    19          0.140744    0.091418  0.049326
                    20          0.075709    0.047095  0.028614
                    21          0.082897    0.050330  0.032568
                    22          0.090990    0.063741  0.027249
                    23          0.074641    0.046437  0.028204
                    24          0.056452    0.057099 -0.000647
                    25          0.100126    0.060568  0.039558
                    26          0.121781    0.068833  0.052948
                    27          0.123933    0.089004  0.034929
                    29          0.182576    0.157837  0.024739
                    30          0.076461    0.049906  0.026556
                    31          0.056489    0.057136 -0.000647
                    32          0.083302    0.054308  0.028993
                    33          0.076293    0.049795  0.026497
                    35          0.085145    0.057927  0.027218
...                                  ...         ...       ...
2015-05-01 23:00:00 117         0.337438    0.500388 -0.162949
                    118         0.112363    0.056633  0.055729
                    119         0.069164    0.046249  0.022914
                    120         0.088230    0.049978  0.038252
                    121         0.069065    0.046183  0.022881
                    122         0.065908    0.042767  0.023141
                    123         0.044944    0.021025  0.023919
                    124         0.044944    0.021025  0.023919
                    125         0.043758    0.020164  0.023594
                    126         4.286123    3.212515  1.073608
                    128         0.026045    0.012453  0.013592
                    129         0.158295    0.071910  0.086385
                    130         0.026162    0.012509  0.013653
                    131         0.097918    0.053347  0.044572
                    133         0.044908    0.021008  0.023899
                    134         0.092834    0.052586  0.040248
                    135         0.045453    0.032652  0.012801
                    136         0.045292    0.032536  0.012756
                    137         0.028819    0.018195  0.010624
                    138         0.026157    0.012507  0.013651
                    139         0.743785    0.624667  0.119118
                    140         0.066431    0.033369  0.033062
                    141         0.332351    0.492843 -0.160493
                    142         0.103077    0.066234  0.036844
                    143         0.249177    0.172496  0.076681
                    145         0.066423    0.033365  0.033058
                    149         0.185359    0.144229  0.041129
                    150         0.057243    0.039933  0.017309
                    151         0.067918    0.040145  0.027773
                    152         0.096457    0.061060  0.035397

[3264 rows x 3 columns]

In [136]:
trips.round()

arrivals  departures  flow
date_hour           station_id                            
2015-05-01 00:00:00 3                0.0         0.0   0.0
                    4                0.0         0.0   0.0
                    5                0.0         0.0   0.0
                    6                0.0         0.0   0.0
                    7                0.0         0.0  -0.0
                    8                0.0         0.0   0.0
                    9                0.0         0.0   0.0
                    10               0.0         0.0   0.0
                    11               0.0         0.0   0.0
                    12               0.0         0.0   0.0
                    13               0.0         0.0   0.0
                    14               0.0         0.0   0.0
                    15               0.0         0.0   0.0
                    16               0.0         0.0   0.0
                    17               0.0         0.0   0.0
                    19               0.0         0.0   0.0
                    20               0.0         0.0   0.0
                    21               0.0         0.0   0.0
                    22               0.0         0.0   0.0
                    23               0.0         0.0   0.0
                    24               0.0         0.0  -0.0
                    25               0.0         0.0   0.0
                    26               0.0         0.0   0.0
                    27               0.0         0.0   0.0
                    29               0.0         0.0   0.0
                    30               0.0         0.0   0.0
                    31               0.0         0.0  -0.0
                    32               0.0         0.0   0.0
                    33               0.0         0.0   0.0
                    35               0.0         0.0   0.0
...                                  ...         ...   ...
2015-05-01 23:00:00 117              0.0         1.0  -0.0
                    118              0.0         0.0   0.0
                    119              0.0         0.0   0.0
                    120              0.0         0.0   0.0
                    121              0.0         0.0   0.0
                    122              0.0         0.0   0.0
                    123              0.0         0.0   0.0
                    124              0.0         0.0   0.0
                    125              0.0         0.0   0.0
                    126              4.0         3.0   1.0
                    128              0.0         0.0   0.0
                    129              0.0         0.0   0.0
                    130              0.0         0.0   0.0
                    131              0.0         0.0   0.0
                    133              0.0         0.0   0.0
                    134              0.0         0.0   0.0
                    135              0.0         0.0   0.0
                    136              0.0         0.0   0.0
                    137              0.0         0.0   0.0
                    138              0.0         0.0   0.0
                    139              1.0         1.0   0.0
                    140              0.0         0.0   0.0
                    141              0.0         0.0  -0.0
                    142              0.0         0.0   0.0
                    143              0.0         0.0   0.0
                    145              0.0         0.0   0.0
                    149              0.0         0.0   0.0
                    150              0.0         0.0   0.0
                    151              0.0         0.0   0.0
                    152              0.0         0.0   0.0

[3264 rows x 3 columns]